# Preprocess and Upload Data

In [1]:
import boto3
import pandas as pd

import io
import requests
from PIL import Image
from multiprocessing import Pool

from tqdm import tqdm

In [2]:
s3 = boto3.resource("s3")
bucket = s3.Bucket("ap-unsplash-images")

In [3]:
df = pd.read_csv(
    "data/photos.tsv000",
    sep="\t"
)[[
    "photo_id",
    "photo_image_url",
    "photo_width",
    "photo_height",
    "photo_aspect_ratio",
    "exif_camera_make",
    "exif_camera_model",
    "exif_iso",
    "exif_aperture_value",
    "exif_focal_length",
    "exif_exposure_time",
]].dropna()

df.head()

,photo_id,photo_image_url,photo_width,photo_height,photo_aspect_ratio,exif_camera_make,exif_camera_model,exif_iso,exif_aperture_value,exif_focal_length,exif_exposure_time
0,2Q8zDWkj0Yw,https://images.unsplash.com/photo-141520117961...,4192,2794,1.50,NIKON CORPORATION,NIKON D700,2000.0,2.5,50.0,1/125
1,tsBDNuCJiLg,https://images.unsplash.com/photo-141768928330...,4324,2880,1.50,NIKON CORPORATION,NIKON D3200,320.0,7.1,200.0,1/2000
2,A93gsuMxVcE,https://images.unsplash.com/photo-142981401899...,2000,1333,1.50,Canon,Canon EOS REBEL T2i,400.0,14.0,18.0,1/640
3,oYIdH6bFssk,https://images.unsplash.com/photo-143275722183...,4288,2848,1.51,NIKON CORPORATION,NIKON D5000,250.0,5.6,105.0,1/30
4,wgLPy2YBXuc,https://images.unsplash.com/photo-143205996405...,5312,2988,1.78,SAMSUNG,SAMSUNG-SM-G870A,40.0,2.2,4.8,1/2384


In [4]:
def load_image(url):
    resp = requests.get(url)
    b = io.BytesIO(resp.content)
    return Image.open(b)

def reshape_image(img,shape=(960,960)):
    return img.resize(shape)

def upload_img(img,name):
    img = img.convert("RGB")
    with io.BytesIO() as f:
        img.save(f,"JPEG")
        f.seek(0)
        bucket.upload_fileobj(f,
            f"lite-normalized/unsplash.lite.{name}.JPG")
        
def process_image(url,name=None):
    if name is None: url, name = url
    img = load_image(url)
    img = reshape_image(img)
    upload_img(img,name)

In [ ]:
with Pool() as P:
    P.map(
        process_image,
        df[["photo_image_url","photo_id"]].values
    )